In [10]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt 
from scipy.stats import norm 

In [12]:
ticker = ['SPY', 'AAPL', 'TLT', 'GLD']
weights = np.array([0.5, 0.2, 0.1, 0.2])
initial_asset = 10000000

data = yf.download(ticker, start='2010-01-01', end='2026-01-01')
returns = data.pct_change().dropna()

[*********************100%***********************]  4 of 4 completed


In [15]:
average_returns = returns.mean()
cov_matrix = returns.cov()

print(average_returns)


Price   Ticker
Close   AAPL      0.001089
        GLD       0.000369
        SPY       0.000577
        TLT       0.000152
High    AAPL      0.001047
        GLD       0.000364
        SPY       0.000555
        TLT       0.000146
Low     AAPL      0.001084
        GLD       0.000368
        SPY       0.000572
        TLT       0.000144
Open    AAPL      0.001105
        GLD       0.000368
        SPY       0.000577
        TLT       0.000149
Volume  AAPL      0.051841
        GLD       0.105805
        SPY       0.051970
        TLT       0.088000
dtype: float64


In [ ]:
n_simulations = 10000
days = 1

L = np.linalg.cholesky(cov_matrix)

portfolio_simulations = np.zeros(n_simulations)

for i in range (n_simulations):
    random_noise = np.random.normal(size=(len(ticker), days))
    simulated_return = np.dot(random_noise, L)

    portfolio_returns = np.dot(weights, simulated_return)
    portfolio_simulations[i] = initial_asset * (1+portfolio_returns)